# Fatty Liver Disease (FLD) Study

- alcoholic vs non-alcoholic FLD, short: AFLD vs NAFLD


**Outline**

1. Study on liver disease types:
    1. Fibrosis
    1. Steatosis
    2. Inflammation
    
2. Two data sets with 
    1. clinical markers
    2. proteome information
    
**Highlighted Contents**
> In order to jump to highlighted sections, use a table of contents plugin (toc) for a structured view ([lab](https://github.com/jupyterlab/jupyterlab-toc) | [notebook](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/toc2/README.html)). 
> Without data, but for easier navigation try to go to [colab](https://colab.research.google.com/).

2. Explore datasets
    1. Proteomics data
        - will be published to PRIDE
    2. Clinical data
        - not publically available
3. Models
    1. (3.2) Individual Models for three endpoints fibrosis, steatosis and inflammation
        - Cross-validation results
    2. (3.4) Final Model
        - final model used for DeLong-Test comparison and clinical follow-up evaluation


> Some data is hidden from the public output until it is cleared. `#hide`

In [ ]:
import os
from pathlib import Path
CPUS = os.cpu_count()
RANDOMSTATE = 29

FOLDER_DATA_RAW = 'data/raw'
DATAFOLDER = 'data/processed'
os.makedirs('data/processed', exist_ok=True)
TABLEFOLDER = 'tables'
RESULT_FOLDER = 'results'
FIGURE_FOLDER = Path('figures')
FIGURE_FOLDER.mkdir(exist_ok=True)

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
import sklearn.linear_model as skllm
from src.pandas import combine_value_counts
import sklearn.metrics as sklm
import sklearn.model_selection as sklms
from sklearn.model_selection import cross_val_predict

import ipywidgets as widgets
import tqdm

In [ ]:
import src

# Explore datasets

Diagnostic comparators (existing best-in-class) biomarkers
- Fibrosis markers: transient elastography, 2-dimensional shear wave elastography, ELF test, FibroTest, FIB4 score, APRI score, Forns score, ProC3
- Inflammation markers: M30=caspase-cleaved cytokeratin-18 fragments, M65=total CK18, AST:ALT ratio, ProC3
- Steatosis: Controlled attenuation parameter

In [ ]:
pd.set_option('max_columns', 9)

_folder = FOLDER_DATA_RAW
_index_col = 'Sample ID'

files = [file for file in os.listdir(_folder) if '.csv' in file]

if not files:
    print("No processed files found.")
else:
    w_data = widgets.Dropdown(options=files)

    show_data = src.widgets.create_show_data(index_col=_index_col, datafolder=_folder)
    out = widgets.interactive_output(show_data, controls={'file':w_data})

    data = show_data.__closure__[0].cell_contents
    w_cols = widgets.SelectMultiple(options=list(data.columns))

    show_selected_proteins = src.widgets.create_show_selected_proteins(data=data)

    out_sel = widgets.interactive_output(show_selected_proteins, {'columns': w_cols})
    out_sel = widgets.interactive_output(show_selected_proteins, {'columns': w_cols})

    # Updater
    def widget_updater(other_widget):
        """Picks first element from closure. other_widget is not used directly"""
        data = show_data.__closure__[0].cell_contents
        w_cols.options = list(data.columns)
        show_selected_proteins.__closure__[0].cell_contents = data

    _ = widgets.interactive_output(widget_updater, {'other_widget': w_data})

    display(widgets.VBox([w_data, out, w_cols, out_sel]))

## Olink proteomics data

### Load Complete Olink proteomics data

In [ ]:
doubleIDkey = pd.read_csv('data/raw/DoubleIDkey.csv')
doubleIDkey['Participant ID']=doubleIDkey['Participant ID'].str.replace('SIPHON', 'ALD')
df_olink = pd.read_csv('/Volumes/auditgroupdirs/SUND-CBMR-Liver-Genetics-OUH/proteogenomics/GALA_wide.csv')

In [ ]:
df_olink['Participant ID']=df_olink['SampleID'].str.replace('ALD', 'ALD_')
df_olink['Participant ID']=df_olink['Participant ID'].str.replace('HP', 'HP_')
df_olink['Sample ID']=df_olink['Participant ID'].map(dict(zip(doubleIDkey['Participant ID'], doubleIDkey['Sample ID'])))
df_olink=df_olink.drop(['Unnamed: 0', 'SampleID', 'Participant ID'], axis=1).set_index('Sample ID')

In [ ]:
index_no = df_olink.columns.get_loc('IL10RB')
df_olink_prot = df_olink.iloc[:, index_no:]
df_olink_prot.rename_axis('Protein ID', axis=1, inplace=True)

In [ ]:
df_olink_prot.head()

In [ ]:
df_olink_prot.isnull().sum().sum()

In [ ]:
df_olink_prot.loc[:, df_olink_prot.isna().any()].head()

In [ ]:
df_olink_prot.describe().T.sort_values(by='count', ascending=True)

In [ ]:
data_olink = df_olink_prot.dropna(axis=1)

In [ ]:
data_olink.shape

In [ ]:
proteins_olink = data_olink.columns.tolist()
key_ProteinID_olink=pd.DataFrame(list(zip(proteins_olink, proteins_olink)), columns=['Protein ID', 'Gene names']).set_index('Protein ID')

### Olink proteomics data imputation

In [ ]:
df_olink_prot.describe().loc['mean'].hist()

## Proteomics data

### Load Complete proteomics data

Full preprocessing pipeline

In [ ]:
annotation_file = pd.read_csv(os.path.join(FOLDER_DATA_RAW, 'Experiment annotation file.csv'), index_col = [0])
annotation_file_plasma = annotation_file[annotation_file['Sample type'] == 'Plasma']
annotation_file_plasma.index = pd.Index(annotation_file_plasma.index, dtype=int)
display(annotation_file_plasma.head())
annotation_file_plasma.describe()

The annotation file holds the filename for the the processed raw data by Skyline and some annotation, e.g. the `Sample ID`

In [ ]:
annotation_file_plasma["Sample ID"]

The mapping for Protein ID to the gene ID is given by `report_plasma`

In [ ]:
report_plasma = pd.read_csv(os.path.join(FOLDER_DATA_RAW, '20190620_210717_20190620_P0000005_Lili2Klibrary_Report.csv'), na_values='Filtered')
report_plasma.rename({'PG.Genes': 'Gene names', 'PG.ProteinAccessions': 'Protein ID'}, inplace= True, axis=1)
report_plasma.head() 

Create mapping object (see if proteins are unique -> get function from other project)

In [ ]:
columns_ = ['Protein ID', 'Gene names']
ids_ = report_plasma[columns_].apply(lambda series_: series_.str.split(';'))
ids_.head()

In [ ]:
def length_(x):
    try:
        return len(x)
    except:
        return 0
    
count_groups_proteins = ids_.apply(lambda series_: series_.apply(length_))
count_groups_proteins

In [ ]:
from src.pandas import combine_value_counts
combine_value_counts(count_groups_proteins)

- Protein IDs are always set
- two proteins have no annotations (Gene name count of 0 appears twice)
- the are some protein names which are mapped to the same gene. 
Let have a look at cases where a set of proteins was not mapped uniquely to one gene:

In [ ]:
ids_.loc[count_groups_proteins['Gene names'] != 1]

In [ ]:
from src.imputation import imputation_normal_distribution, log2, NP_LOG_FCT, IMPUTATION_MEAN_SHIFT, IMPUTATION_STD_SHRINKAGE
#imputation_normal_distribution??

In [ ]:
# Report_plasma = pd.read_csv('raw/proteomics/plasma/20190620_210717_20190620_P0000005_Lili2Klibrary_Report.csv')
experimental_columns = annotation_file_plasma['Sample ID']
report_plasma[columns_] = report_plasma[columns_].apply(lambda series_: series_.str.split(';').str[0])

In [ ]:
report_plasma.describe()

In [ ]:
map_filenames_ids = dict(zip(annotation_file['File name'], annotation_file['Sample ID']))

Remove some measurements which are not intensities, but ... ?

In [ ]:
data_plasma_raw = report_plasma.copy()
data_plasma_raw.drop(data_plasma_raw.filter(regex='StrippedSequences').columns, axis=1, inplace = True)

- rename column names to sample ID from annotation file
- set index to proteins

In [ ]:
data_plasma_raw = data_plasma_raw.rename(mapper = map_filenames_ids, axis=1)
IDmapping_UniprotID_to_Genename = dict(zip(data_plasma_raw['Protein ID'], data_plasma_raw['Gene names']))
data_plasma_raw = data_plasma_raw.set_index('Protein ID').drop('Gene names', axis = 1)
data_plasma_raw.shape

In [ ]:
mask = data_plasma_raw.notna().sum(axis=1) > 603 * 0.6
mask.sum()

Filter at protein level for 60% data completeness across all runs

In [ ]:
DATA_COMPLETENESS = 0.6
data_plasma_filtered = data_plasma_raw.dropna(axis=0, thresh = data_plasma_raw.shape[1] * DATA_COMPLETENESS)
# data_plasma_filtered #hide

Check how many the plates which will be discarded have:

In [ ]:
mask_filtered_out = data_plasma_filtered.notna().sum() < 200
data_plasma_filtered.loc[:, list(mask_filtered_out)].describe().loc['count'].astype(int).sort_values()

In [ ]:
data_plasma_raw.loc[:, mask_filtered_out].describe().loc['count'].astype(int).sort_values()

> Cutoff of 118 is next one where another sample would be discarded.

Filter at sample level for a total number of quantified protein groups above 200 (of 290).

In [ ]:
MIN_N_PROTEIN_GROUPS = 200
print(f"Min No. of Protein-Groups in single sample: {MIN_N_PROTEIN_GROUPS}, i.e. a fraction of {MIN_N_PROTEIN_GROUPS/len(data_plasma_filtered)}")

In [ ]:
data_plasma_filtered = data_plasma_filtered.dropna(axis=1, thresh = MIN_N_PROTEIN_GROUPS)
# data_plasma_filtered #hide

In [ ]:
assert (data_plasma_filtered.dtypes != float).sum() == 0

In [ ]:
# data_plasma_filtered = convert_to_numeric(data_plasma_filtered)
# data_plasma_filtered_log = np.log2(data_plasma_filtered)
data_plasma_filtered_log = data_plasma_filtered.apply(log2)
# data_plasma_filtered_log #hide

##### Imputation

- imputation is done before coefficient of variation (CV)
- is this sensible?

In [ ]:
SCALE_DATA = False
if SCALE_DATA:
    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()

    data_plasma_filtered_log_imputed_np = scaler.fit_transform(data_plasma_filtered_log.values)
    data_plasma_filtered_log_imputed = data_plasma_filtered_log.copy()
    data_plasma_filtered_log_imputed.loc[:,:] = np.nan_to_num(data_plasma_filtered_log_imputed_np)
else:
    data_plasma_filtered_log_imputed = data_plasma_filtered_log.apply(imputation_normal_distribution)
    assert data_plasma_filtered_log_imputed.loc['Q9Y6Z7', 'Plate1_A2'] - 9.770809 < 0.0001, 'Imputed value changed in comparison to previous run'

In [ ]:
#ToDo: Look at distribution of imputed values vs non-imputed values by protein.
# create data viewer with overlap?

In [ ]:
from pathlib import Path
file = Path('data/processed/plasma_processed.csv')
file.parent.mkdir(parents=True, exist_ok=True)

In [ ]:
import logging
try:
    data_plasma_filtered_log_imputed.to_csv(file.absolute())
except PermissionError as e:
    logging.warning(f"No write permission to directory: {e}")

In [ ]:
# data_plasma_filtered #hide

In [ ]:
qc_plasma = annotation_file_plasma[annotation_file_plasma['Group2'] == 'QC']['Sample ID']
df_qc = data_plasma_filtered.copy()[qc_plasma]
coef_of_variation = lambda x: np.std(x) / np.mean(x)
proteins_cv = df_qc.apply(coef_of_variation, axis = 1)

In [ ]:
CV_COEFFICIENT = 0.3
cv_selected = proteins_cv < CV_COEFFICIENT
print(f"Selected proteins # {cv_selected.sum()} of a total of # {len(cv_selected)}!")

In [ ]:
df_qc = df_qc.assign(cv = proteins_cv)
qc_30 = df_qc[cv_selected].index

df = data_plasma_filtered_log_imputed.copy()
df = df.rename_axis('Sample ID', axis=1).T
# filter proteins for CV < 30% of the inter-day/plate quality assessment 
df_30 = df[qc_30]
data_proteomics = df_30

In [ ]:
PROTEOM  = 'data_ml_proteomics_cleaned.csv'

data_proteomics.to_csv(os.path.join(DATAFOLDER, PROTEOM))
# data_proteomics #hide

In [ ]:
print("A maximum of {1} proteins in {0} samples can be used for proteomic models".format(*data_proteomics.shape))

**Low intensities** below 8 (in log-scale)

In [ ]:
intensities_below_8 = data_proteomics[data_proteomics < 8].dropna(how='all').dropna(how='all', axis=1)
# intensities_below_8 #hide

In [ ]:
# data_proteomics.loc[intensities_below_8.index, intensities_below_8.columns] #hide

Data proteomics is the summary of the following processing steps:

1. protein is selected if shared betw. 60% of samples
2. sample is selected if it has at least 200 proteins
3. log-transform
4. imputation (imputation done per protein between runs)
5. selection using CV < 0.3

> Maybe create an automated report of the cutoffs.

In [ ]:
summary_protein_preprocessing = [("Proportion protein has to be shared between samples" , DATA_COMPLETENESS),
                                 ("Minimum number of protein in single sample", MIN_N_PROTEIN_GROUPS),
                                 ("Maximum coefficient of variation (CV) for protein intensities", CV_COEFFICIENT),
                                 ("Logarithm employed for transformation", NP_LOG_FCT),
                                 ("Imputation: Mean-Shift", IMPUTATION_MEAN_SHIFT), 
                                 ("Imputation: Std-Dev. shrinkage", IMPUTATION_STD_SHRINKAGE)
                                ]

for descr, value in summary_protein_preprocessing:
    print('{}: {}'.format(descr, value))

### Load Protein GeneID Mapping

- UniProtID to Gene name mapping
- the assigned protein groups are mapped to mainly one, sometimes two genes -> Global Identifiers?!


In [ ]:
key_ProteinID = pd.read_csv(os.path.join(FOLDER_DATA_RAW, 'ID_matching_key.csv'), 
                            index_col="Protein ID").drop("Unnamed: 0", axis=1)
key_ProteinID.head()

In [ ]:
key_ProteinID.loc['A0A075B6R9']

Note that there are possibly alternative protein names, which are mapped to the same gene.

In [ ]:
ids_.head()

In [ ]:
assert len(key_ProteinID) == len(ids_), "Both references should match at least in the number of proteins. "

## Clinical data
### Load Complete clinical data

In [ ]:
CLINICAL = 'df_cli_164.csv'
COL_ID = 'Sample ID'

In [ ]:
CLINICAL = 'df_cli_164.csv'
COL_ID = 'Sample ID'

f_data_clinic = os.path.join(FOLDER_DATA_RAW, CLINICAL)
data_cli = pd.read_csv(f_data_clinic, index_col=COL_ID)
data_cli = data_cli[data_cli['kleiner']!=0.5]
# data_cli #hide

In [ ]:
w_cols_cli = widgets.SelectMultiple(options=list(data_cli.columns))

def show_selected_markers(columns):
    if len(columns)> 0:
        display(data_cli[list(w_cols_cli.value)])
        display(data_cli[list(w_cols_cli.value)].describe())
    else:
        print('Select clinical markers')

out_cli = widgets.interactive_output(show_selected_markers, {'columns': w_cols_cli})
widgets.VBox([w_cols_cli, out_cli])

### Selected Clinical markers

Diagnostic comparators (existing best-in-class) biomarkers
- state-of-the-art (**SOTA**) Fibrosis markers: 
    - `te`: transient elastography (sona liver scan)
    - `swe`: 2-dimensional shear wave elastography
    - `elf`: ELF test
    - `ft`: FibroTest
    - `fib4`: FIB4 score
    - `apri`: APRI score
    - `forns`: Forns score
    - `p3np`: ProC3
- Inflammation markers:
    - M30=caspase-cleaved cytokeratin-18 fragments
    - M65=total CK18
    - AST:ALT ratio
    - ProC3
- Steatosis: Controlled attenuation parameter

In [ ]:
#SOTA_fibrosis = ['te', 'swe', 'elf', 'ft', 'fib4', 'apri', 'forns', 'p3np']
SOTA_fibrosis = ['elf', 'ft', 'fib4', 'apri', 'forns', 'p3np']
data_cli.groupby('kleiner')[SOTA_fibrosis].count()

In [ ]:
pd.set_option('max_columns', 20)
FEATURES_ML = ['nas_steatosis_ordinal', 'nas_inflam', 'kleiner', 
          'fib4', 'elf', 'ft', 'te', 'swe', 'aar','ast',
          'apri','forns','m30', 'm65', 'meld', 'p3np', 'timp1', 'cap' ]
# data_cli[FEATURES_ML].head() #hide

In [ ]:
data_cli.groupby('group2')[FEATURES_ML].count()

In [ ]:
SOTA_fibrosis = ['te', 'swe', 'elf', 'ft', 'fib4', 'apri', 'forns', 'p3np']
data_cli.groupby('kleiner')[SOTA_fibrosis].median()

### Selected Demographics

In [ ]:
demographics = data_cli[['age', 'bmi', 'gender_num']] # 1 is male
demographics.describe()

In [ ]:
SELECTED_DEMOGRAPHICS = ['age', 'gender_num']
data_cli[SELECTED_DEMOGRAPHICS].head()

### Targets

In [ ]:
fibrosis_score = data_cli.kleiner
inflamation_score = data_cli.nas_inflam
steatosis_score = data_cli.nas_steatosis_ordinal

In [ ]:
TARGETS = ['kleiner', 'nas_steatosis_ordinal', 'nas_inflam']
Y = data_cli[TARGETS]
Y.describe()

In [ ]:
from src.pandas import combine_value_counts
#combcombine_value_counts??

freq_targets = combine_value_counts(Y)
freq_targets.loc['Total',:] = freq_targets.sum()
freq_targets.to_excel(os.path.join(TABLEFOLDER, 'freq_endpoints_unique_values_olink.xlsx'))
freq_targets

Several binary features can be created.

target      | Scale   | unique values              | Binarization                 |  N samples
-----       | --------| ---------------            | -------------------------    |  ---------
fibrosis    | five    | F0, F1, F2, F3, F4         | (F0,F1) vs (F2, F3, F4)      |  360
fibrosis    | five    | F0, F1, F2, F3, F4         | (F0,F1,F2) vs (F3, F4)       |  360
inflamation | seven   | I0, I1, I2, I3, I4, I5     | (I0, I1) vs (I2, I3, I4, I5) |  352
steatosis   | five    | S0, S1, S2, S3             | (S0) vs (S1, S2, S3)         |  352


Variable naming: `<target>_greater-equal_<value>`

In [ ]:
from src.pandas import create_dichotome
kleiner_ge_2     = create_dichotome(Y['kleiner'], 2)
kleiner_ge_3     = create_dichotome(Y['kleiner'], 3)
steatosis_ge_1   = create_dichotome(Y['nas_steatosis_ordinal'], 1)
inflamation_ge_2 = create_dichotome(Y['nas_inflam'], 2)

end_points = ['F2', 'F3', 'S1', 'I2']
dichotomies = [kleiner_ge_2, kleiner_ge_3, steatosis_ge_1, inflamation_ge_2]
targets_dict  = {k: v for k, v in zip(end_points, dichotomies)}

Frequencies of binary variables:

In [ ]:
freq_targets = pd.DataFrame(
    {'kleiner>=2': kleiner_ge_2.value_counts(dropna=False, sort=False),
     'kleiner>=3': kleiner_ge_3.value_counts(dropna=False, sort=False),
     'steatosis>=1' : steatosis_ge_1.value_counts(dropna=False, sort=False),
     'inflamation>=2':inflamation_ge_2.value_counts(dropna=False, sort=False)
    })
freq_targets.loc['total'] = freq_targets.sum()
freq_targets

### Clinical Cutoffs for targets

Cutoff for binary grouping of targets

target      | Scale   | unique values            | N samples
----------- | ------- | ----------------------   | -------
fibrosis    | five    | F0, F1, F2, F3, F4       | 360
steatosis   | five    | S0, S1, S2, S3           | 352
inflamation | seven   | I0, I1, I2, I3, I4, I5   | 352


In [ ]:
file_cutoff_clinic = os.path.join(FOLDER_DATA_RAW, "clinical_marker_test_cut-offs.xlsx")
cutoffs_clinic = pd.read_excel(file_cutoff_clinic, sheet_name="cutoffs", index_col='marker')
cutoffs_clinic

In [ ]:
markers_to_drop = []
for marker in cutoffs_clinic.index:
    if marker not in data_cli.columns:
        print(f"{marker}: Missing in clinics data.")
        markers_to_drop.append(marker)

`proc3` is not in data_clinic. drop this from the list of cutoffs! (Cutoff can be learned later)
Rename columns to desired endpoint name.

In [ ]:
if markers_to_drop:
    cutoffs_clinic.drop(labels=markers_to_drop, inplace=True)
cutoffs_clinic.columns = ['F2', 'F3', 'I2', 'S1']
cutoffs_clinic

Extract certain cutoff for binary targets defined by column name:

In [ ]:
cutoffs_clinic['F2'].dropna().to_dict()

See statistics (e.g. median) of SOTA-markers for clinical fibrosis assessment (represented by categories 0 to 4).

In [ ]:
SOTA_fibrosis = ['te', 'swe', 'elf', 'ft', 'fib4', 'apri', 'forns', 'p3np']
data_cli.groupby('kleiner')[SOTA_fibrosis].median()

### Handle missing features of clinical data (Global Missing Pattern)

> No imputation of clinical features for now as only single clinical features are used in "univariate" models. Imputation is only sensible if several types of information are combined. Then one could use [`sklearn.impute.simpleImputer`](https://scikit-learn.org/stable/modules/impute.html)'s default `'mean'` strategy or alternatively one could replace missing values with zeros on the standardised data to zero mean and standard deviation of one.

Features are present to widely different degree. In order to be able to define global splits with the same pattern of missings over the features and targets by samples (here: patients), we define a missing pattern for stratification.

In [ ]:
# FEATURES_CLINIC = ['ggt', 'alt', 'ast', 'alk', 'mcv', 'iga', 'igg', 'leu', 'glc']
FEATURES_CLINIC = cutoffs_clinic.index
data_cli[FEATURES_CLINIC].describe()

We keep only samples for which any target is present. The other could be later used for verification of model prediction in the clinic.

In [ ]:
patient_ids_w_target = data_cli[TARGETS].dropna(how='all').index
print(f"No. of samples without target variable: {len(data_cli) -len(patient_ids_w_target)} ")

We now define the set of variables of which we want to define missingness patterns:

In [ ]:
FEATURES_CLINIC_ALL = list(FEATURES_CLINIC) + SELECTED_DEMOGRAPHICS + TARGETS
data_cli.loc[patient_ids_w_target, FEATURES_CLINIC_ALL].describe().sort_values(by="count", ascending=False, axis=1)

In [ ]:
def ordered_missing_table(data:pd.DataFrame):
    """Order dataframe by data completeness (first column has most features) 
    and then return an encoding of completeness (1 = available, 0 0 not available)"""
       
    data_missing_table = data.notna().astype(int)
    var_ordered_by_completness = list(data.describe().loc['count'].sort_values(ascending=False).index)
    data_missing_table = data_missing_table.sort_values(by=var_ordered_by_completness)[var_ordered_by_completness]
    return data_missing_table.replace(0, pd.NA).convert_dtypes()

print("Used features: {}".format(", ".join(FEATURES_CLINIC_ALL)))
data_cli_missing_table = ordered_missing_table(data=data_cli.loc[patient_ids_w_target, FEATURES_CLINIC_ALL])
data_cli_missing_table = data_cli_missing_table.dropna(how='all', axis=0).dropna(how='all', axis=1)
# data_cli_missing_table #hide

In [ ]:
data_cli_missing_table.describe().loc['count'].astype(int)

Compare both available data for proteomics and clinical features. We will add the availability of proteomics data as another feature to our missingness patterns.

In [ ]:
data_proteomics.isna().any(axis=None)

In [ ]:
in_both = data_proteomics.index.intersection(data_cli_missing_table.index)
samples_wo_proteomics_data = data_cli_missing_table.index.difference(in_both)
print("{} diagnosed patients have no valid proteome measure: {}".format(
    len(samples_wo_proteomics_data), 
    ", ".join(samples_wo_proteomics_data)
))

In [ ]:
HAS_QUANT_PROT = 'has_prot'
data_cli_missing_table[HAS_QUANT_PROT] = pd.Series(1, index=data_proteomics.index)

In [ ]:
data_cli_missing_table.dropna(how='all').describe()

In [ ]:
data_cli_missing_table = ordered_missing_table(data_cli_missing_table)

In [ ]:
data_cli_missing_strings = data_cli_missing_table.fillna(value=0)
data_cli_missing_strings = data_cli_missing_strings.astype(str)
stratifier = data_cli_missing_strings.apply(lambda x : x.str.cat(), axis=1)
# display(stratifier.head()) #hide
stratifier_tab = stratifier.value_counts()
stratifier_tab

We will have to get ride of the singletons (unique value only once observed). Possibly the grouping could be extended to the values up to 5.

In [ ]:
unique_missing_patterns = list(stratifier_tab.index)

def match_observed(seq1, seq2):
    return sum(pos1 == pos2 for pos1, pos2 in zip(seq1, seq2))

assert match_observed("111111110011100001", "111111110011110010") == 15, "Failed"

In [ ]:
stratifier.value_counts().min()

In [ ]:
def update_stratifier(stratifier_var:pd.Series, threshold:int=None, verbose:bool=False):
    """Takes a stratifier variable, and assigns the pattern the less 
    often observed (defined by threshold or the minimum) to the closest other missing pattern.
    Clossness is defined by the number of features which are present/absent for the samples."""
    stratifier_var =stratifier_var.copy()
    stratifier_tab = stratifier_var.value_counts()
    current_minimum = stratifier_tab.min()
    if threshold is not None  and current_minimum >= threshold:
        logger.info("Threshold already reached.")
        return stratifier_var
    unique_missing_patterns = list(stratifier_tab.index)
    list_single_missing_patterns = stratifier_tab[stratifier_tab <= current_minimum].index
    for single_missing_pattern in list_single_missing_patterns:
        if verbose:
            logger.info(f"Find match for: {single_missing_pattern}")
        closest = 0
        for i, other_seq in enumerate(unique_missing_patterns):
            if not other_seq in list_single_missing_patterns:
                relatedness = match_observed(single_missing_pattern, other_seq)
                if relatedness > closest:
                    closest = relatedness
                    best = other_seq
        stratifier_var[stratifier_var == single_missing_pattern] = best
        if verbose:
            logger.info(f"Best match is : {best}")
    if threshold is not None:
        stratifier_tab = stratifier_var.value_counts()
        new_minimum = stratifier_tab.min()
        if new_minimum < threshold:
            stratifier_var = update_stratifier(stratifier_var, threshold=threshold)        
    return stratifier_var
# stratifier = update_stratifier(stratifier).value_counts()
stratifier = update_stratifier(stratifier, threshold=5, verbose=True)
stratifier.value_counts()

Global stratification based on string. It won't be possible to distribute unique cases, which is why they were assigned to their closest papern. Then split models between endpoints are comparable as they are subsets of the global splits. This  garuantess:
1. By endpoint: Metrics for marker on one test set does not contain patients which are in the training set of a different marker.
2. By marker: Metrics for different endpoints on the test set does not contain training samples of a different endpoint.

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

CV_FOLDS = 5
CV_REPEATS = 10

RANDOM_SEED = 123

rskf = RepeatedStratifiedKFold(n_splits=CV_FOLDS, n_repeats=CV_REPEATS, random_state=RANDOM_SEED) 
splits = list(rskf.split(data_cli_missing_table, stratifier))

In [ ]:
cv_train_test_indices = list()
for train_indices, test_indices in splits:
    cv_train_test_indices.append(
     (stratifier.index[train_indices], stratifier.index[test_indices])
    )
cv_train_test_indices[0]

## Visualization of data

Look at UMAPs with labels from disease categories.
  - Does the assigned disease correspond to certain groups
 
For clinical data, on could look at a selection of scatter plots in order to see if it is feasible to separate some groups based on two features.

In [ ]:
#ToDo
#import umap

# Models

Different _experimental_ setups for prediction models will be compared. First, for the target **fibrosis**. Fibrosis is reported on a five-point scale from stage F0 to F4.

ML setup binary    | HP  | F0  | F1  | F2  | F3  | F4
--- | --- | ---    | --- | --- | --- | ---
HP-F0-F2 vs F3-F4  | c   | c   | c   | c   | t   | t    
F0-F2 vs F3-F4 (advanced)    |     | c   | c   | c   | t   | t
F0-F1 vs F2-F4 (significant)    |     | c   | c   | t   | t   | t

In the table, c stands for control  and t for target. The clinical relevance is to distinguish different 
stages of disease. The question is wheater one should include a healthy, untested patient cohort can help building a 
classification model, as e.g. for fibrosis the general prevalence in the population is between 6 to 7 percent. Alternatively a _multi-task model_ with having 5 classes/end-points can be fit.


In addition to fibrosis, the endpoints **steatosis** and **inflamation** can be predicted.

target      | Scale   | unique values              | N samples
-----       | --------| ---------------            | -------
fibrosis    | five    | F0, F1, F2, F3, F4         | 
steatosis   | five    | S0, S1, S2, S3, S4         | 
inflamation | seven   | I0, I1, I2, I3, I4, I5, I6 | 


What is population of interest?
- population at risk
- general population (which we do not have as a "random" sample)


## Preparation: Classifiers and Evaluation Functionality

### Predefined (Tree-based classifiers, SVMs, GLMs)
- Select Classifier by cross-validation using [sklearn functionality](https://scikit-learn.org/stable/model_selection.html#model-selection)

In [ ]:
clf_lr    = skllm.LogisticRegression(random_state=0, solver='liblinear')
clf_lr_key = 'Logistic'

# specify more sklearn classifiers if you need
clf_sklearn = {clf_lr_key: clf_lr}

[Refitting](https://scikit-learn.org/stable/tutorial/basic/tutorial.html#refitting-and-updating-parameters) the same estimator by invocing it `fit`-method overwrites the previously learned weights.

### Custom Threshold-based classification
Create a classifier based on the threshold which is compatible with the basic scikit-learn functionality, see [instructions](https://scikit-learn.org/stable/developers/develop.html)

Example for using the cutoff of Fibrosis >=2 from the cutoff-table:

In [ ]:
cutoffs_clinic.loc['te','F2']

In [ ]:
from src.threshold_classifier import ThresholdClassifier
clf_te = ThresholdClassifier(threshold={'te':7.0})
print(clf_te.threshold)
clf_te.fit(data_cli.fillna(value=0))
y_pred = clf_te.predict(data_cli)
clf_te.predict_proba(data_cli)[:4] # no scores, either 0 or 1 as cutoff is just compared

Note: Having only one feature for threshold classification does make the definiton of a cutoff unnecessary. AUC-ROC statistics are not meaningful withouth scores.

In [ ]:
# from sklearn.utils.estimator_checks import check_estimator
# check_estimator(ThresholdClassifier)

### Confusion Matrix

In [ ]:
from src.scoring import ConfusionMatrix

How to use it, e.g. for using a clinical marker cutoff for fibrosis:

In [ ]:
y_true = data_cli.kleiner > 2.0

# y_pred defined as Threshold-example
cm_f2_te = ConfusionMatrix(y_true, y_pred)
print("As DataFrame:")
display(cm_f2_te.as_dataframe)
print("Plain:\n",cm_f2_te)

### Cross-Validation Procedure

*Note on Cross-Validation Procedure*
- Comparing the performance on random splits of the entire data will lead to overconfident predicitons.
- Performing the Cross-Validation only on a `Train`-split would allow to have a better evaluation on the test dataset. 
- Cutoff calibration would need a validation split


##### Cutoff Specification
> in clinical setting, false-alarms are preferrable than missed detections. Yes we should find a way to customize the cutoffs
> to have a high sensitivity but also decent specificity, but I guess it risks over-tuning on this specific dataset?  
> Can one ramp over and find the optimal based on F1 score? Would MCC be a better alternative?  
> (Author?)

#### Selected Metrics for Binary Classification evaluation

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_curve
scoring = ['precision', 'recall', 'f1', 'balanced_accuracy', 'roc_auc']
scoring

We build a dictionary of the scoring functions for later use:

#### Define population in order to obtain comparable splits of the data

The common functionality provided by `sklearn` does not allow for [nested stratification](https://stackoverflow.com/a/45526792/9684872). `RepeatedKFold` is splitting based on the data in the target variable.
One solution is to encode a sample with missing target or feature values explicitly into the target variable, but this is not feasible for many different sets of feature and target variables (here each marker-variable with each target is a set).

Data has to be aligned for computation. Due to missing values on some features, the runs are not directly comparable.

In [ ]:
_y = kleiner_ge_2
_X = data_cli.te.to_frame().fillna(0)
in_both = _y.index.intersection(_X.index)
_X = _X.loc[in_both]

e.g. for clinical marker

In [ ]:
from src.cross_validation import run_cv_binary_simple, _get_cv_means
clf = {**{'f2_te': clf_te}, **clf_sklearn}
print("Klassifiers:", ", ".join(clf.keys()))

result_dict = run_cv_binary_simple(clf, X=_X, y=_y, cv=5, scoring=scoring, return_estimator=True)
result_dict.keys()

In [ ]:
_get_cv_means(result_dict).sort_values(('test_f1', 'mean'))

> NOTE: The ROC_AUC value is misleading in case of the ThresholdClassification `f2_te` as the predictor does not yield probabilites ($y_{predicted} \in \{0,1\}$).

#### Try to visualize Decision

##### Univariate Logistic Regression
For the univariate logistic regression
$$ ln \frac{p}{1-p} = \beta_0 + \beta_1 \cdot x $$
the cutoff `c=0.5` corresponds a feature value of: 
$$ x = - \frac{\beta_0}{\beta_1} $$

In [ ]:
for lr_est in result_dict['Logistic']['estimator']:
    # lr_0 = result_dict['Logistic']['estimator'][0]    
    print(f"Custom cutoff defined by Logistic regressor: {- float(lr_est.intercept_) / float(lr_est.coef_):.2f} ")

#### Rebuilded `run_cv_binary` to get roc_curve value

The re-implemented interface for `run_cv_binary` has a a similar interface as sklearns [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html). The `group` parameter is missing as it's not used currently in this setup.

In [ ]:
from src.cross_validation import run_cv_binary
#run_cv_binary?

In [ ]:
results_dict, roc_curve_results, precision_recall_results = run_cv_binary(clf, X=_X, y=_y, cv=cv_train_test_indices, prefix='F2_', verbose=True)

In [ ]:
# mean over flattend array
assert np.mean(pd.DataFrame(results_dict).loc['y_test', 'F2_Logistic']) == np.mean(np.array(pd.DataFrame(results_dict).loc['y_test', 'F2_Logistic']).flatten())

Display CV results (metrics):

In [ ]:
_get_cv_means(results_dict)

#### Averaging the predictions
Get predictions for samples as average of the predictions on test set

In [ ]:
_df = pd.DataFrame(index=_y.index)
for _i, _y_pred in enumerate(results_dict['F2_Logistic']['y_test']):
    _df[f'run_{_i:02}'] = _y_pred
display(_df.head())
results = _df.mean(axis=1).to_frame(name='mean')
results['std'] = _df.std(axis=1, skipna=True)
results['n_pred'] = _df.notna().sum(axis=1).astype(int)
results.head()

In [ ]:
del results

#### Confidence-Intervals

In [ ]:
result_metrics = _get_cv_means(results_dict)
result_metrics

In [ ]:
def create_95CI_df(df_metrics, selected_metrics=None):
    """Expects output from _get_cv_means."""
    if selected_metrics is None:
        selected_metrics = result_metrics.columns.levels[0]
    else:
        assert set(selected_metrics) in set(result_metrics.columns.levels[0])
    
    key_lower_CI = 'lower'
    key_upper_CI = 'upper'
    
    def _create_95CI_df(df_metric, mean_col='mean', std_col='std'):
        """Create from a DataFrame of results the 95% CI.
        Lower and upper bound."""
        df_CI = pd.DataFrame(index=df_metric.index)
        df_CI[key_lower_CI] = df_metric[mean_col] - 2*df_metric[std_col]
        df_CI[key_upper_CI] = df_metric[mean_col] + 2*df_metric[std_col]
        return df_CI

    df_95CI = pd.DataFrame(index=df_metrics.index, columns=pd.MultiIndex.from_product([selected_metrics, ['lower', 'upper']]))
    for _metric in selected_metrics:
        df_95CI[_metric] = _create_95CI_df(df_metrics[_metric])
    return df_95CI

create_95CI_df(result_metrics)

#### Extension: Learn imputation on fold
Include Preprocessing (here: imputation into the pipeline). The imputation of the proteomics data would then be based only on moments learned the training data (splits) for the Gaussian distribution of each peptide.

> write custom [`FunctionTransformer`](https://scikit-learn.org/stable/modules/preprocessing.html#custom-transformers) to included preprocessing.


In [ ]:
# #ToDo: Possible extension
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

def custom_preprocessing_function(X):
    """Operate on a a set of rows from the dataset.
    here: apply imputation to log-transformed values?
    """
    raise NotImplementedError

# clf = make_pipeline(FunctionTransformer(custom_preprocessing_function), svm.SVC(C=1))
# cross_val_score(clf, X, y, cv=cv)

## Individual Models for three endpoints fibrosis, steatosis and inflammation
Strategy for feature selection and model performance validation: 3 models to be built, fibrosis (F0-1 vs. F2-4, and F0-2 vs. F3-4), inflammation (0-1 vs. 2-5) and steatosis (0 vs. >0). Then compare each model with their respective existing best-in-class markers according to their standard cut-offs in clinic. 

In [ ]:
from src.sklearn import FeatureSelector
    
feature_selected = FeatureSelector(k=10, protein_gene_data=key_ProteinID_olink)
_proteins_selected_f2 = feature_selected.fit(data_olink, kleiner_ge_2, 'F2')
_proteins_selected_f2.columns = ['F2 (k=10)']
_proteins_selected_f2['F2 (k=5)'] = feature_selected.get_k_best('F2', 5)
_proteins_selected_f2

### Screen for optimized number of features

[Feature selection](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection) based on mutual information

In [ ]:
from sklearn.exceptions import UndefinedMetricWarning
import warnings; warnings.simplefilter('ignore', UndefinedMetricWarning)

RECALCULATE_FEATURES = False
RESULT_FEATURE_COMPARISON = os.path.join(DATAFOLDER, 'summary_n_features_olink.pkl')
def main_n_features_comparison(n_features_max=88):
    "compare performance using an grid of features"      
    from tqdm.notebook import tqdm as tqdm
    from time import perf_counter as pc
    t0 = pc()
    summary = []
    feature_selected = FeatureSelector(k=n_features_max, protein_gene_data=key_ProteinID_olink)
    _ = feature_selected.fit(data_olink, kleiner_ge_2, 'F2')
    _ = feature_selected.fit(data_olink, kleiner_ge_3, 'F3')
    _ = feature_selected.fit(data_olink, steatosis_ge_1, 'S1')
    _ = feature_selected.fit(data_olink, inflamation_ge_2, 'I2')
    for n_features in tqdm(range(1,n_features_max)): 
#         feature_selected = FeatureSelector(k=n_features, protein_gene_data=key_ProteinID)
        proteins_selected_f2 = feature_selected.get_k_best('F2', n_features)
        proteins_selected_f3 = feature_selected.get_k_best('F3', n_features)
        proteins_selected_s1 = feature_selected.get_k_best('S1', n_features)
        proteins_selected_I2 = feature_selected.get_k_best('I2', n_features)
        test_cases = {}
        test_cases['F2'] = {'proteins': proteins_selected_f2, 'y':kleiner_ge_2}
        test_cases['F3'] = {'proteins': proteins_selected_f3, 'y':kleiner_ge_3}
        test_cases['S1'] = {'proteins': proteins_selected_s1, 'y':steatosis_ge_1}
        test_cases['I2'] = {'proteins': proteins_selected_I2, 'y':inflamation_ge_2}
        for test_case in test_cases.keys():
            _clf_key = 'LR'
            _clf = skllm.LogisticRegression(random_state=0, solver='liblinear')
            proteins_selected = test_cases[test_case]['proteins']
            y = test_cases[test_case]['y']
            _X = data_olink[proteins_selected.index]
            in_both = y.index.intersection(_X.index)
            _X = _X.loc[in_both]
            _y = y.loc[in_both]
#             result = cross_validate(_clf, X=_X, y=_y, groups=_y, cv=RepeatedStratifiedKFold(n_splits=CV_FOLDS, n_repeats=CV_REPEATS, random_state=RANDOM_SEED) , scoring=scoring)
            result, _, _ = run_cv_binary({_clf_key:_clf}, X=_X, y=_y, cv=cv_train_test_indices, prefix=f'{test_cases["F2"]}_', verbose=False)
            _key = list(result.keys()).pop()
            result = pd.DataFrame(result[_key])
            result['name'] = _clf.__class__.__name__
            result['n_features'] = n_features
            result['test_case'] = test_case
            summary.append(result)
    summary = [pd.DataFrame(_) for _ in summary]
    summary_n_features = pd.concat(summary)
    summary_n_features.to_pickle(RESULT_FEATURE_COMPARISON) # long format
    print(f"Finished. Elapsed seconds {pc()-t0:.2f}")
    return summary_n_features

if not RECALCULATE_FEATURES:
    try:
        summary_n_features = pd.read_pickle(RESULT_FEATURE_COMPARISON)
    except FileNotFoundError:
        summary_n_features = main_n_features_comparison()
else:
    summary_n_features = main_n_features_comparison() 

In [ ]:
plt.figure(figsize=(5,5))
sns.lineplot(x='n_features',y='roc_auc',hue='test_case', data=summary_n_features)
plt.ylim([0.5,1])
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Number of Features vs roc auc')
plt.show()

In [ ]:
summary_n_features

In [ ]:
fig=plt.figure(figsize=(16.5, 4))
metrics = ['f1', 'balanced_accuracy', 'roc_auc']
for i in range(3):
    plt.subplot(1, 3, i+1)
    sns.lineplot(x='n_features',y=metrics[i],hue='test_case', data=summary_n_features[summary_n_features['test_case']!='F3'], palette=['darkblue', 'gray', 'darkred'])
    plt.ylim(0, 1)
    #plt.title('Number of Features vs {}'.format(metrics[i]), fontsize=14)
    plt.ylabel(metrics[i], fontsize=14)
    plt.xlabel('Number of features', fontsize=14)
    plt.xticks(fontsize=14);
    plt.yticks(fontsize=14);
    plt.ylim(0.6, 1);
plt.savefig('figures/1_vs_panel_olink.png', dpi=120, bbox_inches='tight')

In [ ]:
combined = summary_n_features.groupby(['test_case','n_features']).mean()

best = combined.sort_values(by='f1', ascending=False).groupby('test_case').head(1)
best

In [ ]:
combined

In [ ]:
single_protein = combined[combined['num_feat']==1]
single_protein

In [ ]:
one_vs_panel = pd.concat([best, single_protein]).sort_values(by='test_case')
#one_vs_panel.to_csv('tables/1_vs_panel.csv')
one_vs_panel

In [ ]:
best_dict = {}
for i, j in best.index.to_list():
    best_dict[i] = j
best_dict

### Top k selected proteins for prediction
[Feature selection](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection) based on mutual information. 

Each endpoint will yield different `top-k` proteins. An aggregation strategy in the simplest form is to combine the top-k. Maybe there is also some kind of rank-algorithm combining the top-k minimizing the overall rank?

In [ ]:
protein_panels={}

for end_point, dichotomy in targets_dict.items():
    k=best_dict[end_point]
    feature_selected = FeatureSelector(k=k, protein_gene_data=key_ProteinID_olink)
    protein_panels[end_point] = feature_selected.fit(data_olink, dichotomy, end_point)

    
proteins_selected_f2=protein_panels['F2']
proteins_selected_f3=protein_panels['F3']
proteins_selected_s1=protein_panels['S1']
proteins_selected_I2=protein_panels['I2']

In [ ]:
proteins_selected_f2.T

In [ ]:
proteins_selected_f3.T

In [ ]:
proteins_selected_s1.T

In [ ]:
proteins_selected_I2.T

In [ ]:
# don't label by gene to retain scoring information (to sort at least by endpoint for importance)
protein_panels_scores={}
for end_point, dichotomy in targets_dict.items():
    k=best_dict[end_point]
    feature_selected = FeatureSelector(k=k)
    protein_panels_scores[end_point] = feature_selected.fit(data_olink, dichotomy, end_point)

In [ ]:
def get_feature_comp(protein_panel:dict, exclude=[], order=None):
    """Custom processor for dictonary holding multual information DataFrame per endpoint
        from above."""
    for i, (endpoint, _df) in enumerate(protein_panel.items()):
        if endpoint not in exclude:
            if i == 0:
                df_protein_panel = _df
            else:
                df_protein_panel = df_protein_panel.join(_df, how='outer')
    if order:
        df_protein_panel = df_protein_panel[order]
    mask = df_protein_panel.isna()   
    # df_protein_panel.where(mask, other=1).fillna(0).sort_values(by=list, ascending=False)
    return df_protein_panel.sort_values(by=list(df_protein_panel.columns), ascending=False).fillna('-')

df_protein_panel = get_feature_comp(protein_panel=protein_panels_scores, exclude=['F3'], order=['F2', 'I2', 'S1'])
df_protein_panel['Gene Name'] = key_ProteinID_olink.loc[df_protein_panel.index]

In [ ]:
pd.set_option('precision', 3)
display(df_protein_panel.head(10))

In [ ]:
df_protein_panel.to_csv('data/processed/protein_panels_olink.csv')

In [ ]:
len(df_protein_panel)

### mrmr feature selction

In [ ]:
from mrmr import mrmr_classif

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold, cross_validate
from sklearn.linear_model import LogisticRegression
from mrmr import mrmr_classif

RESULT_FEATURE_COMPARISON_MRMR = os.path.join(DATAFOLDER, 'summary_n_features_olink_mrmr.pkl')
RECALCULATE_FEATURES = False

def n_features_comparison_mrmr(n_features_max=50):
    summary = []
    test_cases = {}
    test_cases['F2'] = {'proteins': proteins_selected_f2, 'y':kleiner_ge_2}
    test_cases['F3'] = {'proteins': proteins_selected_f3, 'y':kleiner_ge_3}
    test_cases['S1'] = {'proteins': proteins_selected_s1, 'y':steatosis_ge_1}
    test_cases['I2'] = {'proteins': proteins_selected_I2, 'y':inflamation_ge_2}    
    for test_case in test_cases.keys():
        cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
        model = LogisticRegression(random_state=42, solver='liblinear')
        scoring = ['precision', 'recall', 'f1', 'balanced_accuracy', 'roc_auc']
        X = data_olink
        y = test_cases[test_case]['y']
        in_both = y.index.intersection(X.index)
        _X = X.loc[in_both]
        _y = y.loc[in_both]
        for n_features in range(1, n_features_max):
            selected_features = mrmr_classif(_X, _y, K=n_features)
            _X_mrmr = _X[selected_features]
            scores = cross_validate(model, _X_mrmr, _y, scoring=scoring, cv=cv)
            scores['n_features'] = n_features
            scores['test_case'] = test_case
            scores['n_observations'] = _X.shape[0]
            results = pd.DataFrame(scores)
            summary.append(results)
    summary = [pd.DataFrame(_) for _ in summary]
    summary_n_features = pd.concat(summary)
    summary_n_features.to_pickle(RESULT_FEATURE_COMPARISON_MRMR)
    return(summary_n_features)

if not RECALCULATE_FEATURES:
    try:
        summary_n_features_mrmr = pd.read_pickle(RESULT_FEATURE_COMPARISON_MRMR)
    except FileNotFoundError:
        summary_n_features_mrmr = n_features_comparison_mrmr()
else:
    summary_n_features_mrmr = n_features_comparison_mrmr()
    

In [ ]:
combined_mrmr = summary_n_features_mrmr.groupby(['test_case','n_features']).mean()

In [ ]:
plt.figure(figsize=(5,5))
sns.lineplot(x='n_features',y='test_roc_auc',hue='test_case', data=summary_n_features_mrmr)
plt.ylim([0.5,1])
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Number of Features vs. ROC-AUC', fontsize=12)
plt.xlabel('Number of features', fontsize =12)
plt.ylabel('ROC-AUC', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('figures/olink_mrmr.png', bbox_inches='tight', dpi=120)

#### Top k mrmr selected proteins for prediction 

- Based on roc auc plot shown above, the minimal number of features to achieve a "good enough" performance for 
-- F2 is 4 
-- S1 is 12 
-- I2 is 2

In [ ]:
test_cases = {}
test_cases['F2'] = {'n_features': 4, 'y':kleiner_ge_2}
test_cases['S1'] = {'n_features': 12, 'y':steatosis_ge_1}
test_cases['I2'] = {'n_features': 2, 'y':inflamation_ge_2} 

mrmr_markers = {}
for test_case in test_cases.keys():
    X = data_olink
    y = test_cases[test_case]['y']
    in_both = y.index.intersection(X.index)
    _X = X.loc[in_both]
    _y = y.loc[in_both]
    markers = mrmr_classif(_X, _y, K=test_cases[test_case]['n_features'])
    markers = pd.DataFrame({i: markers for i in ['Protein ID', test_case]}).set_index('Protein ID')
    mrmr_markers[test_case]=markers

In [ ]:
mrmr_markers['I2']

### Executor 

corresponds to main function in a script. Allows changes over all endpoints simultaneously. 

In [ ]:
from src.cross_validation import MainExecutorCV
cv_executor = MainExecutorCV(proteomics_data=data_olink, clinical_data=data_cli, demographics=demographics, clf_sklearn=clf_sklearn, cutoffs_clinic=cutoffs_clinic)

1. Models based on clinical marker thresholds (Clinical Reference Models) - defined by `cutoffs_clinic`
2. Additional Models based on clinical markers (having no standard cutoffs defined or if data dependent cutoff is wanted) - defined by list `additional_markers`
    - As this depends on the endpoint, it is 
3. Proteomics Models based on protein intensities

Performance depends on the number of available features (varies!) 

Result tables:
- metrics are reported for the test set
- `N_obs` is the number of patients both in the training and testing data set which is split into 80% training and 20% testing splits.

In [ ]:
ADD_DEMOGRAPHICS = False
INTERACTION_DEGREE = 1
VERBOSE = False

### Fibrosis

#### F2

In [ ]:
kleiner_ge_2.value_counts(dropna=False)

In [ ]:
cutoffs_f2 = cutoffs_clinic['F2'].dropna().to_dict()
cutoffs_f2

In [ ]:
y = kleiner_ge_2.astype(int)
y.describe()

In [ ]:
f2_results, f2_auc_scores, f2_prc_scores = cv_executor.run_evaluation(y=kleiner_ge_2,
                                                       endpoint='F2',
                                                       additional_markers=['forns', 'p3np'],
                                                       proteins_selected=mrmr_markers['F2'],
                                                       add_demographics=ADD_DEMOGRAPHICS,
                                                       interactions_degree=1,
                                                       cv=cv_train_test_indices,
                                                       verbose=VERBOSE)
result_table_f2 = _get_cv_means(f2_results).sort_values(('f1', 'mean'), ascending = False)
result_table_f2

In [ ]:
assert result_table_f2.loc['F2_prot_Logistic', ('roc_auc', 'mean')] - 0.8812105411992736 < 0.00001, "Final results not reproduced."

#### F3

In [ ]:
kleiner_ge_3.value_counts(dropna=False)

In [ ]:
cutoffs_f3 = cutoffs_clinic['F3'].dropna().to_dict()
cutoffs_f3

Two clinical markers have no cutoff defined in the literature. Therefore we have to learn these

In [ ]:
f3_results, f3_auc_scores, f3_prc_scores = cv_executor.run_evaluation(y=kleiner_ge_3, endpoint='F3', 
                                                       additional_markers=['p3np'], 
                                                       proteins_selected=proteins_selected_f3,
                                                       add_demographics=ADD_DEMOGRAPHICS,
                                                       interactions_degree=1,
                                                       cv=cv_train_test_indices,
                                                       verbose=VERBOSE)
result_table_f3 = _get_cv_means(f3_results).sort_values(('f1', 'mean'), ascending = False)
result_table_f3

Using one of the models (or the ensemble), one could expect some predictions of fibrosis patients in the untested healthy patient (hp) cohort.

### Inflamation

In [ ]:
inflamation_ge_2.value_counts(dropna=False)

In [ ]:
cutoffs_i2 = cutoffs_clinic['I2'].dropna().to_dict()
cutoffs_i2

In [ ]:
i2_results, i2_auc_scores, i2_prc_scores = cv_executor.run_evaluation(y=inflamation_ge_2, endpoint='I2', 
                                                       additional_markers=['m30', 'm65', 'alt', 'ast', 'm30m65_ratio'], 
                                                       proteins_selected=mrmr_markers['I2'],
                                                       add_demographics=ADD_DEMOGRAPHICS,
                                                       interactions_degree=1,
                                                       cv=cv_train_test_indices,
                                                       verbose=VERBOSE)
result_table_i2 = _get_cv_means(i2_results).sort_values(('f1', 'mean'), ascending = False)
result_table_i2

### Steatosis

In [ ]:
steatosis_ge_1.value_counts(dropna=False)

In [ ]:
cutoffs_s1 = cutoffs_clinic['S1'].dropna().to_dict()
cutoffs_s1

In [ ]:
y = steatosis_ge_1.astype(int)
y.describe()

In [ ]:
s1_results, s1_auc_scores, s1_prc_scores = cv_executor.run_evaluation(y=steatosis_ge_1, endpoint='S1', 
                                                       additional_markers=[], 
                                                       proteins_selected=mrmr_markers['S1'],
                                                       add_demographics=ADD_DEMOGRAPHICS,
                                                       interactions_degree=1,
                                                       cv=cv_train_test_indices,
                                                       verbose=VERBOSE)
result_table_s1 = _get_cv_means(s1_results).sort_values(('f1', 'mean'), ascending = False)
result_table_s1

### Write results to Excel

In [ ]:
FILE_RESULTS = os.path.join(TABLEFOLDER, 'CV_results_olink.xlsx')

with pd.ExcelWriter(FILE_RESULTS) as writer:
    result_table_f2.to_excel(writer, sheet_name='F2_featureOptim')
    result_table_f3.to_excel(writer, sheet_name='F3_featureOptim')
    result_table_i2.to_excel(writer, sheet_name='I2_featureOptim')
    result_table_s1.to_excel(writer, sheet_name='S1_featureOptim')

## Plot Results of Cross validation for three endpoints (F2, I2, S1)

- create [enumeration of subplots](https://stackoverflow.com/a/25544329/9684872) starting at a)

In [ ]:
map_names = pd.read_csv(os.path.join(FOLDER_DATA_RAW, 'naming_scheme.csv'), index_col='name_in_clinical_data')
map_names = map_names['name_in_plot'].to_dict()
map_names

Custom function to transform index names

In [ ]:
display(result_table_s1)

def _process_names(index, map_names=map_names):
    """Helper function for custom labeling of models.
    This function is specific to any dataset and has to be rewritten.
    
    Parameters
    ----------
    index: pandas.Index
        Index to transform. Index names are composite word
        combined with '_' here.
    map_names: dict
        Mapping of names to apply to words.
    """
    names = list(index)
    names = [x.split('_') for x in names]
    endpoint = names[0][0]
    
    def _process_index_names(_l:list):
        REMOVE = 'marker'
        if REMOVE in _l:
            _l.remove(REMOVE)
        _l = [word if not word in map_names else map_names[word] for word in _l]
        CHANGE = {'Logistic': 'Model',
                  'prot': 'Olink'}
        _l = [word if not word in CHANGE else CHANGE[word] for word in _l]
        if CHANGE['Logistic'] not in _l:
            _l.append('Test')
        return _l
    
    for _l in names: assert endpoint == _l[0] , f"Mixed endpoints: {endpoint} and {_l[0]}"
    names = [" ".join(_process_index_names(_l[1:])) for _l in names]
    return names

_process_names(result_table_s1.index)

### Performance Plots based on results DataFrame for a endpoint

In [ ]:
_process_names

In [ ]:
from src.plots import plot_performance
fig, ax = plt.subplots(figsize=(10,10))
plot_performance(ax, result_table_s1, 'balanced_accuracy', 'Steatosis', _process_index=_process_names)

### AUC-ROC Curves based on CV result for an endpoint

In [ ]:
from src.plots import plot_roc_curve

fig, ax = plt.subplots(figsize=(8, 8))      
        
plot_roc_curve(ax, roc_curve_results['F2_Logistic'], 'TARGET')

### Precision Recall Curve

In [ ]:
from src.plots import plot_prc_curve
fig, ax = plt.subplots(figsize=(8, 8))
plot_prc_curve(ax, precision_recall_results['F2_Logistic'], 'TARGET')

### Build final figure for publication

In [ ]:
result_table_f3

In [ ]:
result_table_f2

In [ ]:
import string
fig, axs = plt.subplots(3,3,figsize=(20,20))

n=0
result_tuples = [
 (result_table_f2, f2_auc_scores, 'Fibrosis F2-F4', 'F2_prot_Logistic'), 
 (result_table_i2, i2_auc_scores, 'NAS Inflamation $\geq 2$', 'I2_prot_Logistic'), 
 (result_table_s1, s1_auc_scores, 'NAS Steatosis $\geq 5$%', 'S1_prot_Logistic'), 
    
]

for col, (result_table, result_auc_scores, endpoint_title, auc_model_name) in enumerate(result_tuples):
    
    ax = axs[0,col]
    plot_roc_curve(ax, result_auc_scores[auc_model_name], endpoint_title)
    _ = ax.text(-0.5, 1.1, f"{string.ascii_lowercase[n]})", transform=ax.transAxes, 
                size=20, weight='bold')
    n+=1
    
    ax = axs[1,col]
    plot_performance(ax, result=result_table, metric='f1', title=endpoint_title,  _process_index=_process_names)   
    _ = ax.text(-0.5, 1.1, f"{string.ascii_lowercase[n]})", transform=ax.transAxes, 
                size=20, weight='bold')
    n+=1
    
    ax = axs[2,col]
    _ = ax.text(-0.5, 1.1, f"{string.ascii_lowercase[n]})", transform=ax.transAxes, 
                size=20, weight='bold')
    plot_performance(ax, result=result_table, metric='balanced_accuracy', title=endpoint_title, _process_index=_process_names)   

    n+=1

fig.tight_layout()
fig.savefig(FIGURE_FOLDER / 'Model_performance_olink_mrmr.png', dpi=120, pad_inches=0.1, bbox_inches='tight')

- plot for model ≥ F3

## Final model

1. Either pick one of the models run during CV
2. Aggregate metrics over all CV runs for obs in test set (~ mean of CV results)
3. Perform new train/test split.


Steps to implement

1.  Select model with median aucroc performance
2.  Report summary statistics (mean, median, min, max)
3.  [DeLong](https://github.com/llniu/roc_comparison)
4.  Target Scores for three endpoints of final prediction model


### Check Cross Validation results (for comparison)

#### Look at descriptive statistics of CV

- depending on the split the performance varies. The `min`, `max`, `mean`, etc. are given per model, therefore it's not the given split. Selecting a split of the data which supports one's conclusion can be misleading.

In [ ]:
results = {
 "F2":f2_results,
 "F3":f3_results,
 "I2":i2_results,
 "S1":s1_results
}

In [ ]:
results_combinded = {}
for _results in results.values(): results_combinded.update(_results)
results_combinded.keys()

In [ ]:
def show_summary(results_dict, metric='f1', sort=True, save=False):
    _df = pd.DataFrame(results_dict).loc[metric].apply(pd.Series).T.describe()
    _df.index.name = metric
    if sort:
        _df = _df.sort_values(by='mean', axis=1, ascending=False)
    if save:
        fname = "cv_stats_{}{}_olink.xlsx".format(metric, '_sorted' if save else '')
        fname = os.path.join(TABLEFOLDER, fname)
        _df.to_excel(fname)
        print(f'Saved Table to: {fname}')
    return _df

In [ ]:
out= widgets.interact(show_summary, metric=scoring, results_dict=widgets.fixed(results_combinded))

#### Select a model from the CV run (and compare it to others)

In [ ]:
l_models = list(results_combinded.keys())
ref_model=l_models[14] # 'F2_prot_Logistic'
metric=scoring[2]      # 'f1'

In [ ]:
def compare_models(ref_model:str, metric:str, data:dict, fct=np.median):
    """Select comparison. The first column is the referenc model.
    If the metric of the summary statistic is present several times in the model, this value is returned."""
    ref_model_metric_values = np.array(data[ref_model][metric])
    ref_model_metric = fct(ref_model_metric_values)
    ref_model_metric_idx = (np.abs(ref_model_metric_values - ref_model_metric)).argmin() #ToDo: return both closest values for median.
    ref_model_metric = ref_model_metric_values[ref_model_metric_idx]
    matches = [index for index, item in enumerate(ref_model_metric_values) if item == ref_model_metric]

    _results = pd.DataFrame(ref_model_metric_values[matches], columns=[ref_model], index=matches)
    _results.index.name = 'run'
    
    _selected_results = {}
    
    
    for _model, _result in data.items():
        if not _model == ref_model:
            _selected_results[_model] = [_result[metric][item] for item in matches]
    #sorting of values over last result
    _other = pd.DataFrame(_selected_results, index=matches)
    _other = _other.sort_values(by=matches[-1], axis=1, ascending=False)
    return _results.join(_other)

# compare_models(ref_model=ref_model, metric='precision', data=f2_results, fct=lambda x: np.quantile(x, q=0.6))

In [ ]:
protein_model_name = {endpoint: f'{endpoint}_prot_Logistic'for endpoint in end_points}
metrics_np_fct = {'median': np.median, '3rd quintile': lambda x: np.quantile(x, q=0.6), 'mean': np.mean, 'max': np.max, 'min': np.min }

#needs global dictionaries: protein_model_name, metrics_np_fct, results
def _caller_comp(metric, endpoint, selector):
    """Helper function to use with ipykernel"""
    df = compare_models(ref_model=protein_model_name[endpoint], metric=metric, data=results[endpoint], fct=metrics_np_fct[selector])
    df.columns.name = selector
    return df
# _caller_comp('f1', 'I2', 'median')

In [ ]:
out2= widgets.interact(_caller_comp, metric=scoring, endpoint=results.keys(), selector=metrics_np_fct.keys())

### Train a new final model

Adjust `run_cv_binary`

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(data_cli_missing_table, stratifier, test_size=0.2, stratify=stratifier, random_state=42)
logging.info(f"N train: {len(y_train)}")
logging.info(f"N test: {len(y_test)}")

In [ ]:
combine_value_counts(pd.DataFrame({'train': y_train, 'test': y_test}))

In [ ]:
train_test_split_final = [(X_train.index, X_test.index)] # list of (train, test) indices. Does not throw a nice error message otherwise.

Adapted the executor to take as argument a different function.

In [ ]:
arguments = {}
arguments['F2'] = {'target': kleiner_ge_2, 'proteins': proteins_selected_f2, 'add_markers': ['forns', 'p3np']} 
arguments['F3'] = {'target': kleiner_ge_3, 'proteins': proteins_selected_f3, 'add_markers': ['p3np']}
arguments['I2'] = {'target': inflamation_ge_2, 'proteins': proteins_selected_I2, 'add_markers': ['m30', 'm65', 'alt', 'ast', 'm30m65_ratio']}
arguments['S1'] = {'target': steatosis_ge_1, 'proteins': proteins_selected_s1, 'add_markers': []}

In [ ]:
ADD_DEMOGRAPHICS = False
FOLDER_FINAL_SCORES = 'final_model_scores'

import importlib; importlib.reload(src.cross_validation)
from src.cross_validation import run_cv_binary
prediction_folder = os.path.join(TABLEFOLDER, FOLDER_FINAL_SCORES)


from functools import partial
run_cv_binary_final = partial(run_cv_binary, save_predictions=True, folder=prediction_folder)


results_final = {}

for endpoint, args in arguments.items():
    results_final[endpoint] = cv_executor.run_evaluation(y=args['target'],
                                                       endpoint=endpoint,
                                                       additional_markers=args['add_markers'],
                                                       proteins_selected=args['proteins'],
                                                       evaluator_fct=run_cv_binary_final,
                                                       add_demographics=ADD_DEMOGRAPHICS,
                                                       interactions_degree=1,
                                                       cv=train_test_split_final,
                                                       verbose=False)

In [ ]:
def display_result(endpoint):
    return pd.DataFrame(results_final[endpoint][0]).applymap(lambda x: x[0]).T.sort_values('f1', ascending=False)


# pd.DataFrame(results_final['F2'][0]).T.sort_values('f1', ascending=False)
out3= widgets.interact(display_result, endpoint=results_final.keys())

Save results

In [ ]:
results_final_df = {endpoint: display_result(endpoint) for endpoint in results_final.keys()}

RESULTS_FINAL_MODEL = os.path.join(TABLEFOLDER ,'final_model_results_olink.xlsx')

with pd.ExcelWriter(RESULTS_FINAL_MODEL) as writer:
    for sheet_name, _df in results_final_df.items():
        _df.to_excel(writer, sheet_name=sheet_name)

#### DeLong-Test on final model

First a check of the [implementation](https://github.com/yandexdataschool/roc_comparison) by using a toy example derived in detail in Rachel Draelos' [blog-post](https://glassboxmedicine.com/2020/02/04/comparing-aucs-of-machine-learning-models-with-delongs-test/), where she also references the original DeLong paper from 1988 and a paper on a fast implementation described by Xu Sun and Weichao Xu from 2014.

In [ ]:
import numpy as np
from roc_comparison import compare_auc_delong_xu

ground_trouth = np.array([0,0,1,1,1])
pred_model_a  = np.array([0.1,0.2,0.6,0.7,0.8])
pred_model_b  = np.array([0.3,0.6,0.2,0.7,0.9])

log10_pvalue = compare_auc_delong_xu.delong_roc_test(ground_truth=ground_trouth,
                                      predictions_one=pred_model_a,
                                      predictions_two=pred_model_b
                                 )
assert np.round(10**log10_pvalue[0][0], 4) ==  0.3173 

The implementation currently returns an array of an array with a single float. This could be change to an normal array or a plain floating point number:

In [ ]:
log10_pvalue[0][0], log10_pvalue[0] # one of the two

Get list of dumped results from models.

In [ ]:
folder_final_scores = os.path.join(TABLEFOLDER, FOLDER_FINAL_SCORES)
l_scores = [_csv for _csv in os.listdir(folder_final_scores) if 'Logistic.csv' in _csv]
l_scores

Calculate the p-value based on the common subset of samples between models:

In [ ]:
# #view defintion of imported function
#calc_p_value_delong_xu??

Compare all models between all endpoints and highlight the results

In [ ]:
from src.delong import calc_p_value_delong_xu

model_1 = l_scores[0]
model_2 = l_scores[8]

print(
f"Delong-Test p-value between scores of model {model_1.split('.',1)[0]} and {model_2.split('.')[0]}: "
f"{calc_p_value_delong_xu(model_1=model_1, model_2=model_2, folder_dumps=folder_final_scores, verbose=True):.4f}")

In [ ]:
auc_comp_dict = {}

def highlight_significant(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for floats smaller
    equal to 0.05
    """
    color = 'red' if val <= 0.05 else 'black'
    return 'color: %s' % color

for endpoint in end_points:
    # models = [x for x in l_scores if endpoint in x]
    # model_names = [" ".join(x.split('.csv')[0].split('_')) for x in models]
    model_names = pd.DataFrame(results_final[endpoint][0]).applymap(lambda x: x[0]).T.sort_values('f1', ascending=False).index.to_list()
    model_names = [model for model in model_names if 'Logistic' in model]
    models = [f'{name}.csv' for name in model_names]
    _df = pd.DataFrame(0, index=models, columns=models)
    
    for i, model_1 in enumerate(models):
        for model_2 in models[i:]:
            if model_1 == model_2:
                _df.loc[model_1, model_2] = 1.0
            else:
                _auc_p_value = calc_p_value_delong_xu(model_1, model_2, folder_dumps=folder_final_scores)
                _df.loc[model_1, model_2] = _auc_p_value
                _df.loc[model_2, model_1] = _auc_p_value
    model_names = [" ".join(x.split('.csv')[0].split('_')) for x in model_names]
    _df.columns = model_names
    _df.index   = model_names
    _df =  _df.style.applymap(highlight_significant)
    display(_df)
    auc_comp_dict[endpoint] = _df

Save results for supplementary materials:

In [ ]:
FILE_DELONG = Path(TABLEFOLDER) / 'compare_delong_olink.xlsx'

with pd.ExcelWriter(FILE_DELONG) as writer:
    auc_comp_dict['F2'].to_excel(writer, sheet_name='F2')
    auc_comp_dict['I2'].to_excel(writer, sheet_name='I2')
    auc_comp_dict['S1'].to_excel(writer, sheet_name='S1')

## Final Model evaluation

The healthy patients in the study - have been selected based on age and gender range of the non-healthy patients. These have an `NaN` assigned on the three endpoints.
Another group has been previously excluded from the analysis and can be used here as an pseudo external dataset 

> Note: Pseudo as for now all proteomics pre-processing is done on the entire dataset. Either the procedure is done individuelly for each subset of patients, 
or the mean and std deviation from the training dataset are used to sample random values for missing protein intensities on the test data. The standardisation of the 
proteins intensities after imputation should ideally also be based on the statistics from the training data (which is assumed to the "global" value").

Patiens with an assigned fibrosis score of `0.5` are known to be heavy drinkers without being diagnosed with a fibrosis.

In [ ]:
data_cli_full = pd.read_csv(f_data_clinic, index_col=COL_ID)
# previous selection: data_cli[data_cli['kleiner']!=0.5] 
all_kleiner_score = data_cli_full.kleiner.value_counts(dropna=False)
all_kleiner_score

In [ ]:
#n_healty, n_at_risk = all_kleiner_score.loc[[np.nan, 0.5]]
#One patient in the disease cohort had a 'NaN' kleiner score
n_healthy = data_cli[data_cli['group']=='HP'].shape[0]
n_at_risk = data_cli_full[data_cli_full['kleiner']==0.5].shape[0]
print(f"N Healthy (selected to match ill patients: {n_healthy}")
print(f"N Unhealthy behaviour, but not sick: {n_at_risk}")

In [ ]:
# from src.pandas import combine_value_counts
combine_value_counts(data_cli_full[TARGETS], dropna=False)

In [ ]:
combine_value_counts(data_cli_full.loc[data_cli_full.kleiner == 0.5, TARGETS], dropna=False)

In [ ]:
data_cli_full.loc[data_cli_full.kleiner == 0.5, TARGETS]

#### Load final models

In [ ]:
from joblib import load
endpoints = ['F2', 'I2', 'S1']
fname_final_model = '{}_prot_Logistic.joblib'
final_prot_model = {}
for endpoint in endpoints :
    _fname = os.path.join('tables', FOLDER_FINAL_SCORES, fname_final_model.format(endpoint))
    print(f"Load model from : {_fname}")
    final_prot_model[endpoint] = load(_fname)
    print(final_prot_model[endpoint].coef_)

Protein by endpoint can be retrieved using the previously defined dictionary

In [ ]:
arguments['S1']['proteins'].T

In [ ]:
from src.final_model import FinalPredictor
# FinalPredictor??

In [ ]:
final_predictor = FinalPredictor(data_clinic=data_cli_full, 
                                 data_proteomics=data_olink, 
                                 final_models=final_prot_model, 
                                 features_dict=arguments, 
                                 endpoints=endpoints)

### Feature Importance of final models

- how important are the single proteins, assessed by the model weights.

In [ ]:
# protein panel. Some order?
d_model_weights = {}
d_model_intercepts = {}
for _endpoint, _model in final_predictor.final_models.items():
    d_model_weights[_endpoint] = dict(zip(final_predictor.features_dict[_endpoint]['proteins'].index.to_list(), _model.coef_[0]))
    d_model_intercepts[_endpoint] = _model.intercept_[0]
del _endpoint, _model

model_weights = pd.DataFrame(d_model_weights)

model_weights.index =  [f'{_index}_{_gene}' for _index, _gene 
 in key_ProteinID_olink.loc[model_weights.index].to_dict()['Gene names'].items()
]

model_weights = model_weights.append(pd.Series(d_model_intercepts, name='intercept'))
del d_model_weights, d_model_intercepts
model_weights.to_excel(Path(DATAFOLDER) / 'final_model_weights_olink.xlsx') 
new_index = [i.split('_')[1] for i in model_weights.index[:-1]]
new_index.append('intercept')
model_weights.index=new_index
model_weights=model_weights.sort_values(by='F2', ascending=False).iloc[::-1]
model_weights_F2=model_weights[model_weights['F2'].notnull()][['F2']]
model_weights_I2=model_weights[model_weights['I2'].notnull()][['I2']].sort_values(by='I2')
model_weights_S1=model_weights[model_weights['S1'].notnull()][['S1']].sort_values(by='S1')
model_weights_I2

In [ ]:
ax = model_weights.plot(kind='barh', figsize=(3,15), color=['darkblue', 'darkred', 'gray'])
plt.xticks(fontsize=14);
plt.yticks(fontsize=12);
plt.xlabel('final model weights', fontsize=14);
ax.figure.savefig(FIGURE_FOLDER / 'final_model_weights_olink.png', dpi=120, pad_inches=0.1, bbox_inches='tight')

### At risk population

How many patients at risk have been predicted to have a certain disease stage?

In [ ]:
at_risk_score = final_predictor.predict_score(indices=data_cli_full.kleiner[data_cli_full.kleiner == 0.5].index)
at_risk_pred = final_predictor.predict(indices=data_cli_full.kleiner[data_cli_full.kleiner == 0.5].index)

# minimal check to catch big errors
from numpy.testing import assert_array_almost_equal
assert_array_almost_equal(x=at_risk_score.loc['Plate4_D9'].values,  
                          y=[0.539575564, 0.659919782,0.931215122])

In [ ]:
def highlight_surpassed_score(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for floats smaller
    equal to 0.05
    """
    color = 'yellow' if val >= 0.5 else None
    return f'background-color: {color}'

In [ ]:
at_risk_score = at_risk_score.sort_values(by=list(at_risk_score.columns), ascending=False).style.applymap(highlight_surpassed_score)
at_risk_pred = at_risk_pred.loc[at_risk_score.index]
at_risk_pred.head()

In [ ]:
def build_results_table(predictions:pd.DataFrame, dropna=True):
    """Convert a table of predicitons (for several columns) into a result table."""
    predictions_tab = combine_value_counts(predictions)
    predictions_tab = predictions_tab.join(combine_value_counts(predictions, dropna=dropna) / len(predictions), rsuffix='_freq')
    predictions_tab.loc['Total'] = predictions_tab.sum()
    return predictions_tab.convert_dtypes()

at_risk_results_tab  = build_results_table(at_risk_pred)
at_risk_results_tab

In [ ]:
# ToDo: move to src (replace index building in cross_validation._get_cv_means)
def build_two_level_index(initial_columns, present_key='freq', added_key='prop'):
    """Build a custom multi-index object for data. The initial columns to build the 
    data object are passed along the key describing the intial data and added information by 
    a second key. Could be generalized to work with any number of keys."""
    column_map = []
    for x in initial_columns:
        column_map += [x, x + '_freq']

    levels = [initial_columns, [present_key, added_key]]
    multi_index = pd.MultiIndex.from_product(
        levels, names=['variable', 'statistics'])
    return column_map, multi_index

column_map, multi_index = build_two_level_index(initial_columns=at_risk_pred.columns)
at_risk_results_tab = at_risk_results_tab[column_map]
at_risk_results_tab.columns = multi_index
at_risk_results_tab

### Healthy

How many healthy patients would be predicted to have fibrosis by the final model?

Here we check for formally "healthy" patients (they have not been diagnosed at the initial time of the data collection) and see who would be predicted to 
have a fibrosis score of 2 and above.

We will predict for patients without a fibriosis score. We load the **final** proteomics model from the previous step, lookup the blood plasma protein intensities for these patients on the selected proteins for the classification model, and finally predict their outcome.

In [ ]:
data_cli.fibrosis_class.value_counts(dropna=False)

In [ ]:
data_cli[data_cli['group']=='HP'].shape

In [ ]:
healthy_cohort_mask = data_cli[data_cli['group']=='HP']
healthy_cohort_indices = healthy_cohort_mask.index
healthy_pred = final_predictor.predict(indices=healthy_cohort_indices)
healthy_risk_score = final_predictor.predict_score(indices=healthy_cohort_indices)
healthy_risk_score = healthy_risk_score.sort_values(by=list(healthy_risk_score.columns), ascending=False).style.applymap(highlight_surpassed_score)
healthy_pred = healthy_pred.loc[healthy_risk_score.index]

In [ ]:
healthy_results_tab  = build_results_table(healthy_pred)
column_map, multi_index = build_two_level_index(initial_columns=healthy_pred.columns)
healthy_results_tab = healthy_results_tab[column_map]
healthy_results_tab.columns = multi_index
healthy_results_tab

Of the healty cohort 8 patients are predicted to have a form of advanced fibrosis, which is a share of 5.8% of the patients in this cohort

It is claimed that in the general population a percentage of x has an undiagnosed liver disease. Does our percentage match this?

### Write results

In [ ]:
excel_sheets = {'at_risk_results': at_risk_results_tab,
                'at_risk_pred':at_risk_pred,
                'at_risk_score': at_risk_score,
                'healthy_results': healthy_results_tab, 
                'healthy_pred': healthy_pred,
                'heatlhy_score': healthy_risk_score}
FILE_FINAL_MODEL_EVALUATION = os.path.join(TABLEFOLDER ,'final_model_evaluation_olink.xlsx')

with pd.ExcelWriter(FILE_FINAL_MODEL_EVALUATION) as writer:
    for sheet_name, _df in excel_sheets.items():
        _df.to_excel(writer, sheet_name=sheet_name)

### Extract predicted positive cases and concordance with other non-invasive markers
- F2: swe , te. Cut-offs for ≥F2: 8.6kPa for swe and 7kPa for te
- I2: no equivalent 
- S1: cap. Cut-offs for ≥S1: 290
- Values above cut-offs are color-coded

In [ ]:
try:
    doubleID=pd.read_csv('data/raw/DoubleIDkey.csv', index_col=False)
    display(doubleID)
except:
    doubleID=None

### At risk population

In [ ]:
at_risk_pred_pos_all = at_risk_pred[at_risk_pred.max(axis=1)==1]
data_cli_atrisk = data_cli_full[data_cli_full['kleiner']==0.5]

In [ ]:
at_risk_pos = at_risk_pred_pos_all.join(data_cli_atrisk[['swe', 'te', 'cap']], how='left')
if not doubleID.empty:
    at_risk_pos = at_risk_pos.join(doubleID.set_index('Sample ID'), how='left')
m=at_risk_pos.style.apply(lambda x: ['background: yellow' if v>0 else "" for v in x],
                        subset=['F2', 'I2', 'S1'], axis=1)
m.apply(lambda x:['background: pink' if v>290 else"" for v in x],
             subset=['cap'], axis=0)
os.makedirs(os.path.join('tables', 'validation'), exist_ok=True)
m.data.to_csv('tables/validation/predicted_scores_at_risk_postive_olink.csv')
m

### Healthy population

In [ ]:
healthy_pred_pos_all = healthy_pred[healthy_pred.max(axis=1)==1]

In [ ]:
healthy_pos = healthy_pred_pos_all.join(data_cli[['swe', 'te', 'cap']], how='left')
if not doubleID.empty:
    healthy_pos = healthy_pos.join(doubleID.set_index('Sample ID'), how='left')
l=healthy_pos.style.apply(lambda x: ['background: yellow' if v>0 else "" for v in x],
                        subset=['F2', 'I2', 'S1'], axis=1)
l.apply(lambda x:['background: pink' if v>290 else"" for v in x],
             subset=['cap'], axis=0)
l.data.to_csv('tables/validation/predicted_scores_healthy_postive_olink.csv')
l

### Plot predicted score distribution from the final model in three populations
- healthy population (N=136).
- at risk population (N=98). Heavy alcohol drinkers but benigh liver
- high risk population (N=72). Healvy alcohol drinkers with various forms of liver injuries 

In [ ]:
final_model_test_score = final_predictor.predict_score(indices=train_test_split_final[0][1])
final_model_test_score = final_model_test_score.sort_values(by=list(final_model_test_score.columns), ascending=False)

In [ ]:
model_naming = {'F2':'Fibrosis F2-F4', 'I2':'NAS Inflamation $\geq 2$', 'S1':'NAS Steatosis $\geq 5$%'}
def plot_score_distribution(ax, endpoint):
    ax = sns.kdeplot(data=healthy_risk_score.data[endpoint], label='healthy', color='steelblue', shade=True, ax=ax, cut=0)
    ax = sns.kdeplot(data=at_risk_score.data[endpoint], label='at risk', color='darkorange', shade=True, ax=ax, cut=0)
    ax = sns.kdeplot(data=final_model_test_score[endpoint], label='high risk', color='red', shade=True, ax=ax, cut=0)
    ax.set_xlabel('Risk score predicted by\n proteomic model')
    ax.set_xlim(-0.0, 1.0)
    ax.set_title(model_naming[endpoint])
    return ax

In [ ]:
fig, axes = plt.subplots(ncols=3,figsize=(12,4))
for i, endpoint in enumerate(['F2', 'I2', 'S1']):
    plot_score_distribution(axes[i], endpoint)
fig.savefig(FIGURE_FOLDER / 'Score_kde_all_olink', dpi=120, bbox_inches='tight')

In [ ]:
for endpoint in ['F2', 'I2', 'S1']:
    fig, ax = plt.subplots(figsize=(4,4))
    plot_score_distribution(ax, endpoint)
    fig.savefig(FIGURE_FOLDER / 'Score_kde_{}'.format(endpoint), dpi=120, bbox_inches='tight')

In [ ]:
def plot_score_hist(ax, data, label, color, bins=10, kde=False, rug=True):
    ax = sns.distplot(data, ax=ax, label=label, color=color, bins=bins, kde=kde, rug=rug)
    ax.set_xlabel('Risk score predicted by\n proteomic model')
    ax.set_xlim(-0.0, 1.0)
    ax.set_title(label)
    return 

for endpoint in ['F2', 'I2', 'S1']:
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12,4))
    plot_score_hist(ax=axes[0], data=healthy_risk_score.data[endpoint], bins=10, label='healthy',   color='steelblue', kde=False, rug=True)
    plot_score_hist(ax=axes[1], data=at_risk_score.data[endpoint],      bins=10, label='at risk',   color='darkorange',kde=False, rug=True)
    plot_score_hist(ax=axes[2], data=final_model_test_score[endpoint],  bins=10, label='high risk', color='red',       kde=False, rug=True)
    fig.suptitle(model_naming[endpoint])
    fig.savefig(FIGURE_FOLDER / 'Score_hist_{}_olink'.format(endpoint), dpi=120, bbox_inches='tight')

The rugs (small vertical lines at the botoom of the figure) indicate a single prediction. The scores are devided in 10 bins which means that each bin is decimal range (0.0 to 0.1, 0.1 to 0.2 and so forth).

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12,12))
for i, endpoint in enumerate(['F2', 'I2', 'S1']):
    sns.distplot(healthy_risk_score.data[endpoint], ax=axes[i, 0], bins=10, label='healthy',   axlabel=False, color='steelblue', kde=False, rug=True)
    sns.distplot(at_risk_score.data[endpoint],      ax=axes[i, 1], bins=10, label='at risk',   axlabel=False, color='darkorange',kde=False, rug=True)
    sns.distplot(final_model_test_score[endpoint],  ax=axes[i, 2], bins=10, label='high risk', axlabel=False, color='red',       kde=False, rug=True)

pad = 5 # in point
for i, (_title, _endpoint) in enumerate(zip(['healthy', 'at risk', 'high risk'], ['F2', 'I2', 'S1'])):
    axes[-1, i].set_xlabel('Risk score predicted by\n proteomic model')
    axes[0, i].set_title(_title)
    _ax = axes[i, 0]
    _ax.set_ylabel('frequency')
    _ax.annotate(_endpoint, xy=(0, 0.5), 
                 xytext=(-_ax.yaxis.labelpad - pad, 0),
                xycoords=_ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center')
        
_ = fig.suptitle('Histograms by endpoint', y=.93, fontsize=16 )
fig.savefig(FIGURE_FOLDER / 'Score_hist_all_olink', dpi=120, bbox_inches='tight')

# Python Package Versions

In [ ]:
!pip list | grep pandas

In [ ]:
!pip list | grep scikit

Find the packages in the `requirements.txt` or `environment.yml`

In [ ]:
# %load environment.yml